<p>
  <a href="https://colab.research.google.com/github/ezhilvendhan/ecommerce-demo-gds-vertex-ai/blob/main/similarity.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

# Install Prerequisites
First off, you'll also need to install a few packages.

In [ ]:
%pip install --quiet --upgrade graphdatascience
%pip install --quiet google-cloud-storage

# Working with Neo4j
You'll need to enter the credentials from your Neo4j instance below.  You can get these by running the command ":server connect" in the Neo4j Browser.  The default DB_USER and DB_NAME are always neo4j.

In [ ]:
# Edit these variables!
DB_URL = "neo4j+s://URL:7687"
DB_PASS = ""

# You can leave this default
DB_USER = ''

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS), aura_ds=True)

# Data Import

## Create Constraints

In [ ]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT childCategoryIdConstraint IF NOT EXISTS FOR (c:Category) REQUIRE c.id IS UNIQUE;
    CREATE CONSTRAINT attributeIdConstraint IF NOT EXISTS FOR (a:Attribute) REQUIRE a.id IS UNIQUE;
    CREATE CONSTRAINT productIdConstraint IF NOT EXISTS FOR (p:Product) REQUIRE p.id IS UNIQUE;
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    SHOW CONSTRAINTS YIELD id, name, type, entityType, labelsOrTypes, properties, ownedIndexId;
  """
)
display(result)

## Load Data

Create Nodes

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_node_category.csv' AS line WITH line
    MERGE (n:Category {id:line.id,name:line.name})
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_node_sku.csv' AS line WITH line
    MERGE (n:SKU {id:line.id,name:line.name,attributes:line.attributes,brand:line.brand,colour:line.colour,serial:line.serial})
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_node_keyword.csv' AS line WITH line
    MERGE (n:Keyword {id:line.id,keywords:line.keywords})
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_node_product.csv' AS line WITH line
    MERGE (n:Product {id:line.id,shop_id:line.shop_id,name:line.name})
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_node_attribute.csv' AS line WITH line
    MERGE (n:Attribute {id:line.id,type:line.type,value:line.value})
  """
)
display(result)

Create Relationships

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_relation_IS_CATEGORY.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Category {id:line.category_id})
    CREATE (from)-[:IS_CATEGORY]->(to)
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_relation_IS_SKU.csv' AS line 
    WITH line
    MATCH (from:Product {id:line.product_id}), (to:SKU {id:line.sku_id})
    CREATE (from)-[:IS_SKU]->(to)
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_relation_WITH_KEYWORD.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Keyword {id:line.keyword_id})
    CREATE (from)-[:WITH_KEYWORD]->(to)
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_relation_HOT_SALE.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Product {id:line.product_id})
    CREATE (from)-[:HOT_SALE]->(to)
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_relation_SUPPLEMENT_WITH.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.from_sku_id}), (to:SKU {id:line.to_sku_id})
    CREATE (from)-[:SUPPLEMENT_WITH]->(to)
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/data/import_relation_LOW_PRICE.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Product {id:line.product_id})
    CREATE (from)-[:LOW_PRICE]->(to)
  """
)
display(result)

## Graph Data Science

First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.project(
        'similarity-graph',
        ['Product', 'Sku'],
        '*'
    )
  """
)
display(result)

Note, if you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.drop('similarity-graph')
  """
)
display(result)

Now, let's list the details of the graph to make sure the projection was created as we want.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.list()
  """
)
display(result)

Nowe we can use NodeSimilarity Algorithm to find similar nodes. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/node-similarity/).

In [ ]:
result = gds.run_cypher(
  """
  CALL gds.nodeSimilarity.write('similarity-graph', {
      writeRelationshipType: 'SIMILAR',
      writeProperty: 'score'
  })
  YIELD nodesCompared, relationshipsWritten
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    
  """
)
display(result)

In [ ]:
import pandas as pd
df = pd.DataFrame([dict(record.items()) for record in result['n']])
df

Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [ ]:
embeddings = pd.DataFrame(df['embedding'].values.tolist()).add_prefix("embedding_")
merged = df.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
merged

Now that we have the data formatted properly, let's split it into a training and a testing set and write those to disk.

In [ ]:
df = merged

df['split']=df['reportCalendarOrQuarter']
df['split']=df['split'].replace(['03-31-2021', '06-30-2021', '09-30-2021'], ['TRAIN', 'VALIDATE', 'TEST'])

df = df.drop(columns=['reportCalendarOrQuarter'])

df.to_csv('embedding.csv', index=False)

# Authenticate your Google Cloud Account
Now let's write the file to Google Cloud Storage so we can use it in our model.  To do so, we must first authenticate.

Edit the variables below.  You can find the project ID in the Google Cloud Console.  The STORAGE_BUCKET is the name of a new bucket.  It must be globally unique.  It also needs to be all lower case.

In [ ]:
# Edit this variable!
PROJECT_ID = 'your-project-id'

# You can leave these defaults
STORAGE_BUCKET = PROJECT_ID + '-ev-form13'
REGION = 'us-central1'

In [ ]:
import os
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

In [ ]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Upload to Google Cloud Storage
Now we can upload our data sets to our bucket.

In [ ]:
from google.cloud import storage
client = storage.Client()

In [ ]:
bucket = client.bucket(STORAGE_BUCKET)
bucket.location=REGION
client.create_bucket(bucket)

In [ ]:
filename='embedding.csv'
upload_path = os.path.join('form13', filename)
blob = bucket.blob(upload_path)
blob.upload_from_filename(filename)